## План
- Строим матрицу
- Телепортация
- Нормализуем (сумма строки = 1)
- Домножаем на случайный вектор, пока не сойдется

In [2]:
import os
import urllib
from bs4 import BeautifulSoup
import numpy as np
from tqdm import tqdm

In [3]:
filenames = os.listdir("raw")

In [4]:
len(filenames)

2379

In [5]:
topics = {}
for i, file in enumerate(filenames):
    topics[file[:-5]] = i

In [5]:
matrix = np.zeros((len(filenames), len(filenames)))

In [6]:
for file in tqdm(filenames):
    with open(f"raw/{file}", encoding='utf-8') as f:
        cur_topic = file[:-5]
        bs = BeautifulSoup(f.read(), "html.parser")
        all_a = bs.find_all("a")
        for link in all_a:
            if not 'href' in link.attrs:
                continue
            href = urllib.parse.unquote(link['href'])
            if href.startswith("/wiki/") and href[6:] in topics:
                matrix[topics[cur_topic]][topics[href[6:]]] = 1

100%|██████████| 2379/2379 [01:30<00:00, 26.43it/s]


In [8]:
for i in matrix:
    for j in range(len(i)):
        if i[j] == 0:
            i[j] = 0.001

In [9]:
row_sums = matrix.sum(axis=1)

In [10]:
norm_matrix = matrix / row_sums[:, np.newaxis]

In [11]:
vector = np.array([0.2]*len(filenames))

In [12]:
for _ in range(100):
    vector = norm_matrix.transpose().dot(vector)

In [13]:
np.take(vector, [1, 2])

array([0.90675362, 0.75596967])

In [14]:
rank = {}

In [15]:
for i, value in enumerate(vector):
    rank[i] = value

In [23]:
def page_rank(rank, n=10, word=None):
    if word is not None:
        new_topics = [ind for topic, ind in topics.items() if word.lower() in topic.lower()]
    
    if word is not None:
        new_rank = {}
        for ind in new_topics:
            new_rank[ind] = rank[ind]
    else:
        new_rank = rank.copy()
        
    items = sorted(new_rank.items(), key=lambda t: t[1])[::-1][:n]
    for index, value in items:
        print(f"{filenames[index][:-5]}: {value}")

<img src="data/PageRank.png">

In [17]:
page_rank(rank=rank)

Английский_язык: 6.7102441063494815
США: 4.666826841990531
Франция: 3.891332065960341
Великобритания: 3.755379111713282
Германия: 3.045402505156282
Соединённые_Штаты_Америки: 2.9824853687421458
Италия: 2.411497480410388
СССР: 2.385908911034201
Викисклад: 2.308323432805024
Латинский_язык: 2.305412331063708


In [24]:
page_rank(rank=rank, word='армстронг')

Армстронг,_Луи: 0.21913073924572785
Армстронг,_Нил_Олден: 0.08204167651131537
Армстронг,_Нил: 0.07772736651558698
Армстронг,_Билли_Джо: 0.06033081219855766
Армстронг,_Алан: 0.05467384689837547
Армстронг_(фамилия): 0.054339929130478144
Армстронг,_Эдвин: 0.048724294245225605
Армстронг,_Лэнс: 0.047655826559074285
Билли_Джо_Армстронг: 0.04497463456555407
Армстронг,_Крэйг: 0.03901933675387252


# HITS

In [76]:
new_topics = [ind for topic, ind in topics.items() if 'армстронг' in topic.lower()]

In [77]:
all_topics = set()
for i in new_topics:
    all_topics.add(i)
    for j in range(len(matrix[i])):
        if matrix[i][j] == 1:
            all_topics.add(j)
    for j in range(len(matrix[i])):
        if matrix[j][i] == 1:
            all_topics.add(j)

In [78]:
all_topics = list(all_topics)

In [79]:
h = np.ones(len(all_topics))
a = np.ones(len(all_topics))

In [80]:
for i in range(10):
    new_h = []
    new_a = []
    for j in all_topics:
        row = [matrix[j][t] for t in all_topics]
        cur_h = [in_ * old for in_, old in zip(row, h)]
        cur_h = sum(cur_h)
        new_h.append(cur_h)
        
        out = [matrix[t][j] for t in all_topics]
        cur_a = [out_ * old for out_, old in zip(out, a)]
        cur_a = sum(cur_a)
        new_a.append(cur_a)
        
    new_h_sum = sum(new_h)
    new_h = [cur_h / new_h_sum for cur_h in new_h]
        
    new_a_sum = sum(new_a)
    new_a = [cur_a / new_a_sum for cur_a in new_a]
    
    h = list(new_h)
    a = list(new_a)

In [81]:
h_dict = {}
for i, val in enumerate(h):
    h_dict[i] = val

In [84]:
h_items = sorted(h_dict.items(), key=lambda t: t[1] if all_topics[t[0]] in new_topics else 0)[::-1][:10]

In [89]:
[print(filenames[all_topics[i[0]]]) for i in h_items];

Армстронг,_Лэнс.html
Армстронг,_Нил_Олден.html
Армстронг,_Нил.html
Армстронг,_Рой_Фрейзер.html
Билли_Джо_Армстронг.html
Армстронг,_Билли_Джо.html
Армстронг,_Луи.html
Армстронг,_Роберт.html
Армстронг,_Джерри.html
Армстронг,_Кёрли.html


In [90]:
a_dict = {}
for i, val in enumerate(a):
    a_dict[i] = val

In [93]:
a_items = sorted(a_dict.items(), key=lambda t: t[1] if all_topics[t[0]] in new_topics else 0)[::-1][:10]

In [94]:
[print(filenames[all_topics[i[0]]]) for i in a_items];

Армстронг,_Луи.html
Армстронг,_Нил_Олден.html
Армстронг,_Нил.html
Армстронг,_Билли_Джо.html
Армстронг,_Роберт.html
Армстронг,_Лэнс.html
Армстронг_(округ,_Техас).html
Армстронг_(фамилия).html
Клан_Армстронг.html
Армстронг,_Джерри.html


<img src="data/HITS.png">